In [ ]:
# !pip-compile --strip-extras requirements.in
# %pip install -r requirements.txt

In [3]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

# https://huggingface.co/collections/openai/whisper-release-6501bba2cf999715fd953013
# openai/whisper-tiny
# openai/whisper-tiny.en
# openai/whisper-base
# openai/whisper-base.en
# openai/whisper-small
# openai/whisper-small.en
# openai/whisper-medium
# openai/whisper-medium.en
# openai/whisper-large
# openai/whisper-large-v2
# openai/whisper-large-v3

processor = AutoProcessor.from_pretrained("openai/whisper-base")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-base")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import DataLoader
from torchaudio.datasets import LIBRISPEECH
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Cek device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset LibriSpeech (dev-clean)
train_dataset = LIBRISPEECH("./datasets", url="dev-clean", download=False) # Jika blm punya dataset, bisa set True
test_dataset = LIBRISPEECH("./datasets", url="test-clean", download=False)

In [6]:
from tqdm import tqdm

model.to(device)
model.train()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(5):  # num_train_epochs <= bisa ditambah
    for i, (waveform, sample_rate, transcript, _, _, _) in enumerate(tqdm(train_dataset)):
        # Preprocessing input
        inputs = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt")
        input_features = inputs.input_features.to(device)
        labels = processor.tokenizer(transcript, return_tensors="pt").input_ids.to(device)

        # Forward pass
        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss: {loss.item()}")

  1%|          | 1/124 [00:02<04:46,  2.33s/it]

Epoch 0, Step 0, Loss: 3.035191774368286


  9%|▉         | 11/124 [00:22<03:48,  2.02s/it]

Epoch 0, Step 10, Loss: 0.9028478860855103


 17%|█▋        | 21/124 [00:43<03:34,  2.08s/it]

...

 90%|████████▉ | 111/124 [05:10<00:36,  2.82s/it]

Epoch 4, Step 110, Loss: 0.0325339138507843


 98%|█████████▊| 121/124 [05:40<00:09,  3.10s/it]

Epoch 4, Step 120, Loss: 0.03501022234559059


100%|██████████| 124/124 [05:49<00:00,  2.82s/it]


In [19]:
from datasets import load_metric

# ===============================
# Evaluasi Model pada test_dataset
# ===============================
model.eval()
wer_metric = load_metric("wer")  # Gunakan load_metric dari datasets
cer_metric = load_metric("cer")  # Gunakan load_metric dari datasets

predictions = []
references = []

with torch.no_grad():
    for waveform, sample_rate, transcript, _, _, _ in tqdm(test_dataset):
        inputs = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt")
        input_features = inputs.input_features.to(device)

        predicted_ids = model.generate(input_features)
        predicted_transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        predictions.append(predicted_transcript.lower())  # Semua huruf kecil
        references.append(transcript.lower())  # Semua huruf kecil

# Hitung metrik evaluasi
wer = wer_metric.compute(predictions=predictions, references=references)
cer = cer_metric.compute(predictions=predictions, references=references)

print(f"\nEvaluation Results:")
print(f"WER (Word Error Rate): {wer:.4f}")
print(f"CER (Character Error Rate): {cer:.4f}")


100%|██████████| 15/15 [00:17<00:00,  1.14s/it]


Evaluation Results:
WER (Word Error Rate): 0.3556
CER (Character Error Rate): 0.1462


In [ ]:
import torch

# Simpan model dan tokenizer
model_path = "models/fine_tuned_model.pt"
torch.save(model.state_dict(), model_path)

# Jika ingin menyimpan processor/tokenizer juga
processor.save_pretrained("models/fine_tuned_processor")

[]

In [ ]:
# Muat ulang model dan processor
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-base")
model.load_state_dict(torch.load("models/fine_tuned_model.pt"))
model.to(device)
model.eval()

# Muat processor
processor = AutoProcessor.from_pretrained("openai/whisper-base")

In [ ]:
# # Preprocess the waveform for the model
# inputs = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt")
# input_features = inputs.input_features.to(device)

# # Generate prediction
# model.eval()
# with torch.no_grad():
#     predicted_ids = model.generate(input_features)

# # Decode the prediction
# predicted_transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
# print("Predicted transcript:", predicted_transcript)
# print("Reference transcript:", transcript)

In [ ]:
import gradio as gr

def transcribe(audio_path):
    # Load audio file using torchaudio
    waveform, sr = torchaudio.load(audio_path)
    # Resample if needed
    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
        waveform = resampler(waveform)
    # Preprocess input for the model
    inputs = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt")
    input_features = inputs.input_features.to(model.device)
    # Generate prediction
    model.eval()
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    # Decode the prediction
    predicted_transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return predicted_transcript

# Create Gradio interface with microphone input
gr.Interface(
    title="Whisper ASR",
    description="Automatic Speech Recognition using Whisper",
    fn=transcribe,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs=gr.Textbox(label="Transcription"),
    theme="default",
    allow_flagging="never",
    live=True,
).launch()
